In [1]:
import sys
sys.path.insert(0,'/Users/marcosaponara/Desktop/MY_CENTIPEDE_GAME/CODE/mysrc')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from json import load

from itertools import product
from tqdm import tqdm

from egttools.analytical import PairwiseComparison
from egttools.games import Matrix2PlayerGameHolder
from egttools.utils import calculate_stationary_distribution
from egttools.plotting import draw_invasion_diagram

from scipy.spatial.distance import jensenshannon

from utils.centipedeGame import CentipedeGame
from utils.kStrategy import KStrategy
#
from utils.kernel import eps_kernel_sym as kernel

In [3]:
with open('/Users/marcosaponara/Desktop/MY_CENTIPEDE_GAME/CODE/mysrc/data/data.json') as f:
    data = load(f)

In [4]:
GAMEID = 'icg6'

In [5]:
icgdata = data['data'][GAMEID]

nb_steps = icgdata['nb_steps']
payoffs_pl1 = np.array(icgdata['payoffs_pl1'], dtype = float)
payoffs_pl2 = np.array(icgdata['payoffs_pl2'], dtype = float)
experimental_reference = np.array(icgdata['experimental_reference'], dtype = float)

In [6]:
nb_k_levels = nb_steps + 1
k_levels = np.arange(nb_k_levels)

In [7]:
start = np.zeros(nb_steps+2, dtype = float)

start[-1] = 1.
start[nb_steps//2] = 1.

start_arrays = np.array([
                        start,
                        ], dtype = float)

start_k_pairs = list(product(start_arrays, k_levels))
nb_strategies = len(start_k_pairs)

In [8]:
# Parameters
Z = 1000 # population size

nb_datapoints = 51
nb_eps_datapoints = nb_datapoints
nb_beta_datapoints = nb_datapoints

epsilons = np.linspace(0., .11, nb_eps_datapoints)
betas = np.logspace(-2., 2., nb_beta_datapoints)

gamma = 0.

In [9]:
avg_take_distribution = np.zeros((nb_eps_datapoints, nb_beta_datapoints, nb_steps+1), 
                                 dtype = float)

for j, epsilon in enumerate(tqdm(epsilons)):
    for w, beta in enumerate(betas):
        ker = kernel(nb_steps, gamma, epsilon)
        strategy = KStrategy(ker)

        strategies = []
        for pair in start_k_pairs:
            start, k = pair
            strategies.append(strategy.calculate_mixed_strategy(start = start, k = k))
        strategies = np.array(strategies)

        cg = CentipedeGame(payoffs_pl1, payoffs_pl2, strategies)
        game = Matrix2PlayerGameHolder(nb_strategies, cg.payoffs())

        evolver = PairwiseComparison(population_size=Z, game=game)
        transition_matrix, _ = evolver.calculate_transition_and_fixation_matrix_sml(beta)
        sd = calculate_stationary_distribution(transition_matrix.transpose())

        avg_take_distribution[j,w,:] = sd @ (sd @ cg.get_take_distributions())

  0%|                                                    | 0/51 [00:00<?, ?it/s]/Users/marcosaponara/opt/anaconda3/envs/tomenv/lib/python3.9/site-packages/egttools/utils.py:214: RuntimeWarning: Some of the entries in the transition matrix are close to 1 (with a tolerance of 1e-11). This could result in more than one eigenvalue of magnitute 1 (the Markov Chain is degenerate), so please be careful when analysing the results.
  warn(
100%|███████████████████████████████████████████| 51/51 [04:14<00:00,  5.00s/it]


In [10]:
js = jensenshannon(avg_take_distribution, experimental_reference, axis = -1)

In [11]:
# saving output data
np.save('./' + GAMEID + '_js_grid.npy', js)
np.save('./' + GAMEID + '_beta_array.npy', betas)
np.save('./' + GAMEID + '_eps_array.npy', epsilons)